# Question Paraphrasing

In [1]:
!nvidia-smi
!lscpu

Sat Jul 31 14:36:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0

     |████████████████████████████████| 316 kB 8.1 MB/s 
     |████████████████████████████████| 233 kB 24.9 MB/s 
     |████████████████████████████████| 1.4 MB 38.4 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=03ece3d844d24734d6a891a4d032014c29c26ac3ac4003cdd667fd0e70d3bdbd
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154543 sha256=0e6c9ea324d86cb24d9eb5cb918d013647e3d55fc56940679c820592fda3ea7a
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 43 kB 2.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.

In [3]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Import required packages
import os
import gc
import re
import hazm
import time
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import MT5Config, MT5ForConditionalGeneration, MT5Tokenizer

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
class QuestionParaphrasingDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Question Paraphrasing. """

    def __init__(self, first_questions, second_questions, targets, tokenizer, max_length):
        self.first_questions = first_questions
        self.second_questions = second_questions
        self.tokenizer = tokenizer
        self.targets = targets
        self.max_length = max_length

    def __len__(self):
        return len(self.first_questions)

    def __getitem__(self, item):
        pair = self.first_questions[item] + "<sep>" + self.second_questions[item]
        encoding = self.tokenizer(
            pair,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        inputs = {
            'q1': self.first_questions[item],
            'q2': self.second_questions[item],
            'pair': pair,
            'targets': self.targets[item],
            'input_ids': encoding.input_ids.flatten(),
            'attention_mask': encoding.attention_mask.flatten()
        }
        return inputs


class QuestionParaphrasing:
    def __init__(self, model_name, model_type):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        if model_type.lower() == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(model_name)
            self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
            self.config = MT5Config.from_pretrained(self.model_name)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
            self.id2label = self.config.id2label
            self.label2id = self.config.label2id

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "qqp":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            question_1, question_2, labels, categories = [], [], [], []
            with open(dataset_file, encoding="utf8") as infile:
                for line in infile:
                    json_line = json.loads(line.strip())
                    q1 = json_line['q1']
                    q2 = json_line['q2']
                    label = json_line['label']
                    category = json_line['category']  # qqp or natural
                    if 'category' in kwargs and kwargs['category'] != category:
                        continue
                    question_1.append(q1)
                    question_2.append(q2)
                    labels.append(label)
                    categories.append(category)
            return question_1, question_2, labels, categories

    def mt5_question_paraphrasing_inference(self, input_text_1, input_text_2, device):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        new_input = []
        for q1, q2 in zip(input_text_1, input_text_2):
            new_input.append(q1 + "<sep>" + q2)

        tokenized_batch = self.tokenizer(
            new_input,
            padding=True,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        input_ids = tokenized_batch.input_ids.to(device)
        attention_mask = tokenized_batch.attention_mask.to(device)
        outputs = self.model.generate(input_ids=input_ids,
                                      attention_mask=attention_mask)
        predictions = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return predictions

    def mt5_evaluation(self, input_text_1, input_text_2, input_labels, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(input_text_1) != len(input_text_2):
            print('length of two inputs is not equal!!')
            return
        if len(input_text_1) != len(input_labels):
            print('length of inputs and labels is not equal!!')
            return

        dataset = QuestionParaphrasingDataset(first_questions=input_text_1, second_questions=input_text_2,
                                              targets=input_labels, tokenizer=self.tokenizer, max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#q1:{len(input_text_1)}, #q2:{len(input_text_2)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        total_time = 0
        output_predictions = []
        golden_labels, predicted_labels = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model.generate(input_ids=b_input_ids, attention_mask=b_attention_mask)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')

            b_targets = batch['targets']
            golden_labels.extend(b_targets)

            b_predictions = self.tokenizer.batch_decode(b_outputs, skip_special_tokens=True)
            predicted_labels.extend(b_predictions)

            for i in range(len(b_input_ids)):
                output_predictions.append((batch['q1'][i], batch['q2'][i], b_targets[i], b_predictions[i]))

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(input_text_1))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(
            classification_report(golden_labels, predicted_labels, digits=10)))
        return output_predictions


In [6]:
model_name='persiannlp/mt5-small-parsinlu-qqp-query-paraphrasing'
qp_model = QuestionParaphrasing(model_name=model_name, model_type="mt5")
print(qp_model.config)

You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


MT5Config {
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.7.0",
  "use_cache": true,
  "vocab_size": 250112
}



## Sample Inference

In [7]:
q1_list = [
  "چه چیزی باعث پوکی استخوان می شود؟", 
  "من دارم به این فکر میکنم چرا ساعت هفت نمیشه؟", 
  "دعای کمیل در چه روزهایی خوانده می شود؟", 
  "دعای کمیل در چه روزهایی خوانده می شود؟",
  "شناسنامه در چه سالی وارد ایران شد؟",
  "سیب زمینی چه زمانی وارد ایران شد؟"
]
q2_list = [
  "چه چیزی باعث مقاومت استخوان در برابر ضربه می شود؟", 
  "چرا من ساده فکر میکردم به عشقت پابندی؟", 
  "دعای جوشن کبیر در چه شبی خوانده می شود؟", 
  "دعای جوشن کبیر در چه شبی خوانده می شود؟",
  "سیب زمینی در چه سالی وارد ایران شد؟",
  "سیب زمینی در چه سالی وارد ایران شد؟"
]
qp_model.mt5_question_paraphrasing_inference(q1_list, q2_list, device)

['0', '0', '0', '0', '0', '0']

## QQP Dataset


In [8]:
!git clone https://github.com/persiannlp/parsinlu
!ls parsinlu
!ls parsinlu/data/qqp/

Cloning into 'parsinlu'...
remote: Enumerating objects: 1434, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 1434 (delta 110), reused 139 (delta 82), pack-reused 1252
Receiving objects: 100% (1434/1434), 27.81 MiB | 8.51 MiB/s, done.
Resolving deltas: 100% (913/913), done.
data  LICENSE  README.md  requirements.txt  scripts  src
dev.jsonl  test.jsonl  train.jsonl


### Whole dataset

In [9]:
test_question_1_all, test_question_2_all, test_labels_all, test_categories_all = qp_model.load_dataset_test_file(
    dataset_name="qqp", dataset_file="./parsinlu/data/qqp/test.jsonl")
print(test_question_1_all[:5])
print(test_question_2_all[:5])
print(test_labels_all[:5])
print(test_categories_all[:5])
print(len(test_question_1_all))
print(len(test_question_2_all))
print(len(test_labels_all))
print(len(test_categories_all))

['آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنترل کننده این قلمرو روحانی کیست؟', 'چگونه می توانم تماشای فیلم های پورنو را متوقف کنم؟', 'چه کسانی امام علی را خدا میدانند؟', 'آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایالات متحده باشد؟', 'چگونه می توانم سوالی را در این باره بپرسم؟']
['چه چیزی روح فرد را می شکند؟', 'برای جلوگیری از تماشای کامل پورنو باید چه کاری انجام دهم؟', 'چه کسانی می توانند امام زمان را ببینند؟', 'شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا باشد ، چیست؟', 'چگونه می توانم سوال بپرسم؟']
['0', '1', '0', '1', '0']
['qqp', 'qqp', 'natural', 'qqp', 'qqp']
1916
1916
1916
1916


In [10]:
qp_model.mt5_question_paraphrasing_inference(test_question_1_all[:5], test_question_2_all[:5], device)

['0', '1', '0', '1', '1']

In [11]:
!nvidia-smi
!lscpu

Sat Jul 31 14:39:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    28W /  70W |   2276MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
evaluation_output = qp_model.mt5_evaluation(test_question_1_all, test_question_2_all, test_labels_all, device, max_length=512, batch_size=128)

#q1:1916, #q2:1916
#batch: 15
Start to evaluate test data ...
inference time for step 0: 1.383519490999987
inference time for step 1: 1.3245928099999844
inference time for step 2: 1.3301975680000169
inference time for step 3: 1.3258919330000083
inference time for step 4: 1.3317854500000124
inference time for step 5: 1.3345128570000213
inference time for step 6: 1.3356362100000183
inference time for step 7: 1.3385308550000161
inference time for step 8: 1.3412303720000125
inference time for step 9: 1.3414049290000207
inference time for step 10: 1.344449439999977
inference time for step 11: 1.346521834000015
inference time for step 12: 1.348742776999984
inference time for step 13: 1.3497476169999914
inference time for step 14: 1.3129616419999763
total inference time: 20.08972578500004
total inference time / #samples: 0.010485243102818393
Test Accuracy: 0.7640918580375783
Test Precision: 0.7640005155774416
Test Recall: 0.7640918580375783
Test F1-Score(weighted average): 0.7615327786536144


In [13]:
for q1, q2, true_label, predicted_label in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(q1, q2, true_label, predicted_label))

آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنترل کننده این قلمرو روحانی کیست؟	چه چیزی روح فرد را می شکند؟	0	0
چگونه می توانم تماشای فیلم های پورنو را متوقف کنم؟	برای جلوگیری از تماشای کامل پورنو باید چه کاری انجام دهم؟	1	1
چه کسانی امام علی را خدا میدانند؟	چه کسانی می توانند امام زمان را ببینند؟	0	0
آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایالات متحده باشد؟	شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا باشد ، چیست؟	1	1
چگونه می توانم سوالی را در این باره بپرسم؟	چگونه می توانم سوال بپرسم؟	0	1
قرص مولتی ویتامین مینرال چه فوایدی دارد؟	مولتی ویتامین مینرال چه کاربردی دارد؟	0	1
دانلود اهنگ حال و هوای عالی از دنیا؟	دانلود اهنگ حال و هوای عالی دنیا؟	1	1
آهنگ آخ چه آزار قشنگی تو میتونی بجنگی؟	دانلود آهنگ آخ چه آزار قشنگی تا میتونی بجنگی؟	1	1
مضرات لیست پیوندی چیست؟	برخی از مزایای لیست های پیوندی چیست؟	0	0
دانلود اهنگ ببین زندگی چیه از نظرتو صادق؟	دانلود اهنگ چراغ زندگی از ایرج؟	0	0
چه ویتامین هایی برای پوست مفید است؟	چه ویتامین هایی برای پوست خوب است؟	1	1
چرا موقع غذا خوردن آبریزش بینی دارم؟

In [14]:
output_file_name = "question_paraphrasing_all_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for q1, q2, true_label, predicted_label in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(q1, q2, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Samples with qqp as their category

In [15]:
test_question_1_qqp, test_question_2_qqp, test_labels_qqp, test_categories_qqp = qp_model.load_dataset_test_file(
    dataset_name="qqp", dataset_file="./parsinlu/data/qqp/test.jsonl", category="qqp")
print(test_question_1_qqp[:5])
print(test_question_2_qqp[:5])
print(test_labels_qqp[:5])
print(test_categories_qqp[:5])
print(len(test_question_1_qqp))
print(len(test_question_2_qqp))
print(len(test_labels_qqp))
print(len(test_categories_qqp))

['آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنترل کننده این قلمرو روحانی کیست؟', 'چگونه می توانم تماشای فیلم های پورنو را متوقف کنم؟', 'آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایالات متحده باشد؟', 'چگونه می توانم سوالی را در این باره بپرسم؟', 'مضرات لیست پیوندی چیست؟']
['چه چیزی روح فرد را می شکند؟', 'برای جلوگیری از تماشای کامل پورنو باید چه کاری انجام دهم؟', 'شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا باشد ، چیست؟', 'چگونه می توانم سوال بپرسم؟', 'برخی از مزایای لیست های پیوندی چیست؟']
['0', '1', '1', '0', '0']
['qqp', 'qqp', 'qqp', 'qqp', 'qqp']
478
478
478
478


In [16]:
qp_model.mt5_question_paraphrasing_inference(test_question_1_qqp[:5], test_question_2_qqp[:5], device)

['0', '1', '1', '1', '0']

In [17]:
!nvidia-smi
!lscpu

Sat Jul 31 14:39:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    39W /  70W |   2276MiB / 15109MiB |     22%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
evaluation_output = qp_model.mt5_evaluation(test_question_1_qqp, test_question_2_qqp, test_labels_qqp, device, max_length=512, batch_size=128)

#q1:478, #q2:478
#batch: 4
Start to evaluate test data ...
inference time for step 0: 1.386693053999977
inference time for step 1: 1.3580397700000049
inference time for step 2: 1.3590231339999832
inference time for step 3: 1.012848488000003
total inference time: 5.116604445999968
total inference time / #samples: 0.010704193401673574
Test Accuracy: 0.7196652719665272
Test Precision: 0.7210002199541948
Test Recall: 0.7196652719665272
Test F1-Score(weighted average): 0.7202819665583982
Test classification Report:
              precision    recall  f1-score   support

           0  0.7804054054 0.7700000000 0.7751677852       300
           1  0.6208791209 0.6348314607 0.6277777778       178

    accuracy                      0.7196652720       478
   macro avg  0.7006422631 0.7024157303 0.7014727815       478
weighted avg  0.7210002200 0.7196652720 0.7202819666       478



In [19]:
for q1, q2, true_label, predicted_label in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(q1, q2, true_label, predicted_label))

آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنترل کننده این قلمرو روحانی کیست؟	چه چیزی روح فرد را می شکند؟	0	0
چگونه می توانم تماشای فیلم های پورنو را متوقف کنم؟	برای جلوگیری از تماشای کامل پورنو باید چه کاری انجام دهم؟	1	1
آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایالات متحده باشد؟	شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا باشد ، چیست؟	1	1
چگونه می توانم سوالی را در این باره بپرسم؟	چگونه می توانم سوال بپرسم؟	0	1
مضرات لیست پیوندی چیست؟	برخی از مزایای لیست های پیوندی چیست؟	0	0
اگر از تولید مثل افراد با ضریب هوشی ضعیف جلوگیری کنیم، آیا در نهایت تکامل انسان ها باهوش تر می شوند؟	چقدر معمول است که برای فیزیکدانان نوشتن مقالات آسانتر از خواندن مقالات باشد؟	0	0
تصور کنید که شما یک دانشجو-فضانورد هستید ، که در این پرتاب شرکت کرده بودید. آنچه در هنگام پرواز و بعد از آن تجربه کرده اید را شرح دهید ، وقتی منگلالیان وارد مدار مریخ شد چه احساسی دیدید؟	آیا فضانوردان واقعاً می توانند مانند مریخ در مریخ زنده بمانند؟	0	0
چرا شرکت هایی مانند Rolex و Blackberry هرگز محصولات خود را تبلیغ نمی کنند؟	آیا

In [20]:
output_file_name = "question_paraphrasing_qqp_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for q1, q2, true_label, predicted_label in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(q1, q2, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Samples with natural as their category

In [21]:
test_question_1_natural, test_question_2_natural, test_labels_natural, test_categories_natural = qp_model.load_dataset_test_file(
    dataset_name="qqp", dataset_file="./parsinlu/data/qqp/test.jsonl", category="natural")
print(test_question_1_natural[:5])
print(test_question_2_natural[:5])
print(test_labels_natural[:5])
print(test_categories_natural[:5])
print(len(test_question_1_natural))
print(len(test_question_2_natural))
print(len(test_labels_natural))
print(len(test_categories_natural))

['چه کسانی امام علی را خدا میدانند؟', 'قرص مولتی ویتامین مینرال چه فوایدی دارد؟', 'دانلود اهنگ حال و هوای عالی از دنیا؟', 'آهنگ آخ چه آزار قشنگی تو میتونی بجنگی؟', 'دانلود اهنگ ببین زندگی چیه از نظرتو صادق؟']
['چه کسانی می توانند امام زمان را ببینند؟', 'مولتی ویتامین مینرال چه کاربردی دارد؟', 'دانلود اهنگ حال و هوای عالی دنیا؟', 'دانلود آهنگ آخ چه آزار قشنگی تا میتونی بجنگی؟', 'دانلود اهنگ چراغ زندگی از ایرج؟']
['0', '0', '1', '1', '0']
['natural', 'natural', 'natural', 'natural', 'natural']
1438
1438
1438
1438


In [22]:
qp_model.mt5_question_paraphrasing_inference(test_question_1_natural[:5], test_question_2_natural[:5], device)

['0', '1', '1', '1', '0']

In [23]:
!nvidia-smi
!lscpu

Sat Jul 31 14:39:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    41W /  70W |   2276MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
evaluation_output = qp_model.mt5_evaluation(test_question_1_natural, test_question_2_natural, test_labels_natural, device, max_length=512, batch_size=128)

#q1:1438, #q2:1438
#batch: 12
Start to evaluate test data ...
inference time for step 0: 1.3640759570000114
inference time for step 1: 1.3640611640000202
inference time for step 2: 1.367931337000016
inference time for step 3: 1.3674295950000044
inference time for step 4: 1.3722302430000184
inference time for step 5: 1.3726067650000005
inference time for step 6: 1.3722509739999964
inference time for step 7: 1.3776072249999913
inference time for step 8: 1.3794333119999749
inference time for step 9: 1.3793407950000187
inference time for step 10: 1.382720255999999
inference time for step 11: 0.3293972980000035
total inference time: 15.429084921000054
total inference time / #samples: 0.010729544451321318
Test Accuracy: 0.7788595271210014
Test Precision: 0.782843700062284
Test Recall: 0.7788595271210014
Test F1-Score(weighted average): 0.7759285802012208
Test classification Report:
              precision    recall  f1-score   support

           0  0.7595078300 0.8682864450 0.8102625298    

In [25]:
for q1, q2, true_label, predicted_label in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(q1, q2, true_label, predicted_label))

چه کسانی امام علی را خدا میدانند؟	چه کسانی می توانند امام زمان را ببینند؟	0	0
قرص مولتی ویتامین مینرال چه فوایدی دارد؟	مولتی ویتامین مینرال چه کاربردی دارد؟	0	1
دانلود اهنگ حال و هوای عالی از دنیا؟	دانلود اهنگ حال و هوای عالی دنیا؟	1	1
آهنگ آخ چه آزار قشنگی تو میتونی بجنگی؟	دانلود آهنگ آخ چه آزار قشنگی تا میتونی بجنگی؟	1	1
دانلود اهنگ ببین زندگی چیه از نظرتو صادق؟	دانلود اهنگ چراغ زندگی از ایرج؟	0	0
چه ویتامین هایی برای پوست مفید است؟	چه ویتامین هایی برای پوست خوب است؟	1	1
چرا موقع غذا خوردن آبریزش بینی دارم؟	چرا همش آبریزش بینی دارم؟	0	1
چرا ساعت هوشمند به گوشی وصل نمیشه؟	چرا گوشی هواوی به وای فای وصل نمیشه؟	0	0
چه چیزی باعث مقاومت استخوان در برابر ضربه می شود؟	چه چیزی باعث ورم بدن در دوران بارداری می شود؟	0	0
چه چیزی باعث مقاومت استخوان در برابر ضربه می شود؟	چه چیزی باعث پوکی استخوان می شود؟	0	0
چرا من ساده فکر میکردم به عشقت پابندی؟	من دارم به این فکر میکنم چرا ساعت هفت نمیشه؟	0	0
دعای جوشن کبیر در چه شبی خوانده می شود؟	دعای کمیل در چه روزهایی خوانده می شود؟	0	0
چه کسانی به یارانه م

In [26]:
output_file_name = "question_paraphrasing_natural_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for q1, q2, true_label, predicted_label in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(q1, q2, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()